In [ ]:
import tensorflow as tf
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/MLOM Labs/Lab 5

/content/drive/My Drive/MLOM Labs/Lab 5


**1. Import necessary libraries**

In [ ]:
#Import libraries
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

**2. Load ascii text and covert to lowercas**e

In [ ]:
filename = "data.txt"
raw_text = open(filename, 'r', encoding = 'utf-8').read()
raw_text = raw_text.lower()

In [ ]:
#Getting the first 100 characters of the file read
raw_text[0:100]

"project gutenberg's alice's adventures in wonderland, by lewis carroll\n\nthis ebook is for the use of"

**3. create mapping of unique chars to integers**

In [ ]:
#Create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))  #Sorts the unique chars in order
print(chars)

char_to_int = dict((c , i) for i,c in enumerate(chars))  #Chars mapped to int, i and c is a pair, c -> character and i -> index


['\n', ' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
print(char_to_int)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, "'": 7, '(': 8, ')': 9, '*': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, ';': 26, '?': 27, '@': 28, '[': 29, ']': 30, '_': 31, 'a': 32, 'b': 33, 'c': 34, 'd': 35, 'e': 36, 'f': 37, 'g': 38, 'h': 39, 'i': 40, 'j': 41, 'k': 42, 'l': 43, 'm': 44, 'n': 45, 'o': 46, 'p': 47, 'q': 48, 'r': 49, 's': 50, 't': 51, 'u': 52, 'v': 53, 'w': 54, 'x': 55, 'y': 56, 'z': 57}


**4. Getting the details of the dataset**

In [ ]:
n_chars = len(raw_text)  #Total no. of charcaters
n_vocab = len(chars)     #Total no. of unique charcaters

print("Total characters: ", n_chars)
print("Total Vocab(Unique characters): ", n_vocab)

Total characters:  163780
Total Vocab(Unique characters):  58


In [ ]:
raw_text[0:10]

'project gu'

**5. Prepare the dataset of input to output pairs encoded as integers.**

In [ ]:
#Prepare the dataset of input to output pairs encoded as integers
seq_length = 15 #Can be changed
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[ i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])

n_patterns = len(dataY)
print("Total Patterns: " , n_patterns)

Total Patterns:  163765


In [ ]:
print(dataX[1])
print(dataY[1])

[49, 46, 41, 36, 34, 51, 1, 38, 52, 51, 36, 45, 33, 36, 49]
38


**6. Transform the list of input sequences into the form [samples, time steps, features] that is expected by an
LSTM network and rescale the integers to the range [0,1] to make the patterns easier to learn by the LSTM
network that uses the sigmoid activation function by default.**

In [ ]:
#Reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
#Normalize - rescaling the integer values
X = X / float (n_vocab)
print(X)

[[[0.81034483]
  [0.84482759]
  [0.79310345]
  ...
  [0.77586207]
  [0.56896552]
  [0.62068966]]

 [[0.84482759]
  [0.79310345]
  [0.70689655]
  ...
  [0.56896552]
  [0.62068966]
  [0.84482759]]

 [[0.79310345]
  [0.70689655]
  [0.62068966]
  ...
  [0.62068966]
  [0.84482759]
  [0.65517241]]

 ...

 [[0.55172414]
  [0.56896552]
  [0.79310345]
  ...
  [0.79310345]
  [0.79310345]
  [0.72413793]]

 [[0.56896552]
  [0.79310345]
  [0.89655172]
  ...
  [0.79310345]
  [0.72413793]
  [0.86206897]]

 [[0.79310345]
  [0.89655172]
  [0.87931034]
  ...
  [0.72413793]
  [0.86206897]
  [0.22413793]]]


**7. Convert the output values (single characters converted to integers) into a one hot encoding**

In [ ]:
#One hot encode the output variable
y = np_utils.to_categorical(dataY)
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


**8. Define the LSTM model.**

In [ ]:
#Define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
#It can have 1 or more training samples
model.add(Dropout (0.2))
model.add(Dense (y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="adam")
#Define the checkpoint
filepath="..0.0"
checkpoint = ModelCheckpoint (filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

**9. Fitting the model to data**

In [ ]:
#Change the hyperparameter values and train model
epochs = 10
batch_size = 128

In [ ]:
model.fit(X, y, epochs = epochs, batch_size = batch_size, callbacks = callbacks_list)

Epoch 1/10
1280/1280 [==============================] - ETA: 0s - loss: 2.9841
Epoch 1: loss improved from inf to 2.98408, saving model to weights-improvement-01-2.9841.hdf5
1280/1280 [==============================] - 153s 117ms/step - loss: 2.9841
Epoch 2/10
1280/1280 [==============================] - ETA: 0s - loss: 2.7949
Epoch 2: loss improved from 2.98408 to 2.79489, saving model to weights-improvement-02-2.7949.hdf5
1280/1280 [==============================] - 150s 117ms/step - loss: 2.7949
Epoch 3/10
1280/1280 [==============================] - ETA: 0s - loss: 2.7110
Epoch 3: loss improved from 2.79489 to 2.71096, saving model to weights-improvement-03-2.7110.hdf5
1280/1280 [==============================] - 151s 118ms/step - loss: 2.7110
Epoch 4/10
1280/1280 [==============================] - ETA: 0s - loss: 2.6565
Epoch 4: loss improved from 2.71096 to 2.65651, saving model to weights-improvement-04-2.6565.hdf5
1280/1280 [==============================] - 151s 118ms/step - l

**10. Generate Text with the trained LSTM model**

In [ ]:
#Load the network weights
filename = "weights-improvement-10-2.4040.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
int_to_char = dict((i,c) for i, c in enumerate(chars))

In [ ]:
#Generate a random seed
print(len(dataX))
start = numpy.random.randint(0, len(dataX)-1)
print(start)
pattern = dataX[start]
#datax contains list of patterns
print("Seed: ")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

163765
114432
Seed: 
" too far!" and g "


In [ ]:
#Generate characters
length = 10
final = []
for i in range(length):
  #Reshaping the seed sequence before passing it into the LSTM model
  X = numpy.reshape(pattern, (1, len(pattern), 1))
  #print(x)
  #Normalizing the integer values
  X = X/ float(n_vocab)
  #print(x)
  #Making prediction
  prediction = model.predict(X, verbose=0)
  #Get the predicted value with maximum probability
  index= numpy.argmax(prediction)
  #Convert the predicted integer to char
  result = int_to_char[index]
  #print(result)
  final.append(result)
  #Adding the predicted character to the sequence sequence
  pattern.append(index)
  #Removing the first character from the seed sequence
  pattern = pattern[1:len(pattern)]
print(final)

['h', ' ', 'a', ' ', 'd', 'a', 'n', 't', 'e', 'r']
